# Carregando os dados

Inicialmente, vamos carregar os dados de treino e teste.

In [30]:
import pandas
from IPython.display import Markdown 

train_df = pandas.read_csv("train.csv")
test_df = pandas.read_csv("test.csv")

train_df.describe()

,ano,sequencial_candidato,quantidade_doacoes,quantidade_doadores,total_receita,media_receita,recursos_de_outros_candidatos.comites,recursos_de_pessoas_fisicas,recursos_de_pessoas_juridicas,recursos_proprios,recursos_de_partido_politico,quantidade_despesas,quantidade_fornecedores,total_despesa,media_despesa
count,7622.000000,7.622000e+03,7622.000000,7622.000000,7.622000e+03,7622.000000,7.622000e+03,7.622000e+03,7.622000e+03,7.622000e+03,7.622000e+03,7622.000000,7622.000000,7.622000e+03,7622.000000
mean,2008.120703,9.039623e+10,27.673839,25.239963,1.732199e+05,5456.131027,1.965750e+04,2.315820e+04,7.921689e+04,2.577600e+04,2.541136e+04,130.485174,107.849777,1.554507e+05,1346.643975
std,1.996485,9.893847e+10,116.511888,101.804938,4.444187e+05,14003.313610,1.041636e+05,6.689215e+04,2.514138e+05,1.495312e+05,1.308613e+05,411.039689,333.905884,3.901120e+05,6164.552399
min,2006.000000,1.000100e+04,1.000000,1.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000,1.000000,0.000000e+00,0.000000
25%,2006.000000,1.055500e+04,4.000000,3.000000,2.829208e+03,625.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,6.000000,5.000000,2.689600e+03,352.500000
50%,2010.000000,5.000000e+10,9.000000,8.000000,1.334511e+04,1537.775000,1.125000e+03,1.350000e+03,0.000000e+00,1.121600e+03,0.000000e+00,20.000000,18.000000,1.226735e+04,711.365000
75%,2010.000000,1.900000e+11,23.000000,21.000000,1.046682e+05,4666.247500,6.196375e+03,1.574625e+04,1.725875e+04,9.600000e+03,0.000000e+00,80.000000,68.000000,9.633950e+04,1399.812500
max,2010.000000,2.700000e+11,6997.000000,5800.000000,5.690642e+06,500180.000000,3.664205e+06,1.403049e+06,3.178226e+06,5.525600e+06,2.745700e+06,9932.000000,8359.000000,4.989491e+06,500177.000000


In [31]:
train_df

,ano,sequencial_candidato,nome,uf,partido,quantidade_doacoes,quantidade_doadores,total_receita,media_receita,recursos_de_outros_candidatos.comites,...,quantidade_despesas,quantidade_fornecedores,total_despesa,media_despesa,cargo,sexo,grau,estado_civil,ocupacao,situacao
0,2006,10001,JOSÉ LUIZ NOGUEIRA DE SOUSA,AP,PT,6,6,16600.00,2766.67,0.00,...,14,14,16583.60,1184.54,DEPUTADO FEDERAL,MASCULINO,ENSINO MÉDIO COMPLETO,CASADO(A),VEREADOR,nao_eleito
1,2006,10002,LOIVA DE OLIVEIRA,RO,PT,13,13,22826.00,1755.85,6625.00,...,24,23,20325.99,846.92,DEPUTADO FEDERAL,FEMININO,SUPERIOR COMPLETO,SOLTEIRO(A),SERVIDOR PÚBLICO ESTADUAL,nao_eleito
2,2006,10002,MARIA DALVA DE SOUZA FIGUEIREDO,AP,PT,17,16,158120.80,9301.22,2250.00,...,123,108,146011.70,1187.09,DEPUTADO FEDERAL,FEMININO,SUPERIOR COMPLETO,VIÚVO(A),PEDAGOGO,eleito
3,2006,10002,ROMALDO MILANI,MS,PRONA,6,6,3001.12,500.19,0.00,...,8,8,3001.12,375.14,DEPUTADO FEDERAL,MASCULINO,ENSINO MÉDIO INCOMPLETO,CASADO(A),MILITAR REFORMADO,nao_eleito
4,2006,10003,ANSELMO DE JESUS ABREU,RO,PT,48,48,119820.00,2496.25,0.00,...,133,120,116416.64,875.31,DEPUTADO FEDERAL,MASCULINO,ENSINO FUNDAMENTAL COMPLETO,CASADO(A),DEPUTADO,eleito
5,2006,10003,HELIO BORGES DE SOUSA ESTEVES FILHO,AP,PT,8,8,42668.26,5333.53,2250.00,...,38,37,42668.25,1122.85,DEPUTADO FEDERAL,MASCULINO,SUPERIOR COMPLETO,CASADO(A),ENGENHEIRO,nao_eleito
6,2006,10003,JOSÉ RODRIGUES DOS SANTOS NETO,PI,PCO,6,6,1771.21,295.20,0.00,...,9,9,1771.21,196.80,DEPUTADO FEDERAL,MASCULINO,SUPERIOR COMPLETO,CASADO(A),SERVIDOR PÚBLICO FEDERAL,nao_eleito
7,2006,10003,JOSÉ TOMAZ DA SILVA,MS,PRONA,14,7,1231.00,87.93,0.00,...,17,10,696.91,40.99,DEPUTADO FEDERAL,MASCULINO,SUPERIOR COMPLETO,CASADO(A),MÉDICO,nao_eleito
8,2006,10003,LUIZ CARLOS MUNHOZ MEIRA,MG,PPS,2,2,70000.00,35000.00,0.00,...,332,278,66016.95,198.85,DEPUTADO FEDERAL,MASCULINO,SUPERIOR COMPLETO,CASADO(A),SERVIDOR PÚBLICO FEDERAL,nao_eleito
9,2006,10003,TADEU DE ALENCAR RIBEIRO,DF,PAN,5,3,11570.00,2314.00,0.00,...,51,33,7570.00,148.43,DEPUTADO FEDERAL,MASCULINO,ENSINO MÉDIO COMPLETO,SOLTEIRO(A),APOSENTADO (EXCETO SERVIDOR PÚBLICO),nao_eleito


**1) Há desbalanceamento das classes (isto é, uma classe tem muito mais instâncias que outra)? Em que proporção? Quais efeitos colaterais o desbalanceamento de classes pode causar no classificador? Como você poderia tratar isso?**

Inicialmente, vamos verificar a quantidade de ocorrências para as duas classes (eleito e não-eleito).

In [46]:
qtde_eleito = train_df[train_df["situacao"]  == "eleito"].count()["situacao"]
qtde_nao_eleito = train_df[train_df["situacao"]  == "nao_eleito"].count()["situacao"]

total = qtde_eleito + qtde_nao_eleito
assert total == train_df.count()["situacao"]

perc_eleito = float(qtde_eleito) / float(total) * 100
perc_nao_eleito = float(qtde_nao_eleito) / float(total) * 100
prop_eleito_nao_eleito = float(qtde_eleito) / float(qtde_nao_eleito)
prop_nao_eleito_eleito = float(qtde_nao_eleito) / float(qtde_eleito)


Markdown("Percentual de eleitos = %.2f\n\nPercentual de nao eleitos = %.2f\n\nProporcao entre eleitos e nao eleitos (eleito : nao_eleito) = %.3f\n\nProporcao entre nao eleitos e eleitos (nao_eleito : eleito) = %.3f" % (perc_eleito, perc_nao_eleito, prop_eleito_nao_eleito, prop_nao_eleito_eleito))


Percentual de eleitos = 13.46

Percentual de nao eleitos = 86.54

Proporcao entre eleitos e nao eleitos (eleito : nao_eleito) = 0.156

Proporcao entre nao eleitos e eleitos (nao_eleito : eleito) = 6.429

R: Existe um desbalanceamento entre as classes, onde 13,46% dos dados são da classe "eleito" e 86,54% da classe "nao_eleito", ou seja, para cada não eleito temos 6,429 eleitos. O efeito colateral que o desbalanceamento de classes pode causar é o enviesamento do classificador, onde o mesmo vai ficar propenso a classificar novas instâncias como da classe majoritária. A acurácia não é uma boa métrica para medir a qualidade de um classificador baseado em dados desbalanceados, uma vez que se ele 'chuta' todas as instâncias como sendo da classe majoritária, nesse caso, por exemplo, a acurácia é de 86,54%.
Para mitigar este problema, podemos utilizar classificadores baseados em função de custo bem como utilizar técnicas para sintetização de novas instâncias da classe minoritária ou técnicas de sampling (oversamping, undersampling e hybrid sampling).